# Task 1: Splicing Prediction Training
## Enhanced with Experiment Tracking & Metrics

### 1) Configuration & Setup

In [4]:
import os
import json
import torch
import pandas as pd
from config import (
    DATA_DIR, SAVE_ROOT, BATCH_SIZE, LR, EPOCHS, PATIENCE,
    HIDDEN_DIMS, DROPOUT, NUM_CLASSES, SEED, WEIGHT_DECAY
)

print(f"  Configuration loaded:")
print(f"  Data dir: {DATA_DIR}")
print(f"  Save root: {SAVE_ROOT}")
print(f"  Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")


  Configuration loaded:
  Data dir: D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val
  Save root: d:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models
  Device: cuda


In [5]:
import argparse

# Create argument parser (simulating command-line args in notebook)
parser = argparse.ArgumentParser(description="Task 1 Splicing Prediction Training")
parser.add_argument("--batch-size", type=int, default=2048)
parser.add_argument("--lr", type=float, default=LR)
parser.add_argument('--weight-decay', type=float, default=WEIGHT_DECAY)
parser.add_argument("--epochs", type=int, default=EPOCHS)
parser.add_argument("--patience", type=int, default=PATIENCE)
parser.add_argument("--dropout", type=float, default=DROPOUT)
parser.add_argument("--device", type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
parser.add_argument("--seed", type=int, default=SEED)
parser.add_argument("--data-dir", type=str, default=DATA_DIR)
parser.add_argument("--save-root", type=str, default=SAVE_ROOT)
parser.add_argument("--exp-num", type=int, default=None)
parser.add_argument("--ratio", type=str, default=None, help="Specific ratio (e.g., 'ratio_10_80_10'). If not specified, train all ratios.")
parser.add_argument("--set", type=str, default=None, help="Specific set (e.g., 'set_1'). If not specified, train all sets.")

# Parse empty args to use defaults, or modify as needed
args = parser.parse_args([])

### 2) Train Splicing Models with Experiment Tracking

In [ ]:
from train import main as train_main
# Run training
train_main(args)

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



EXPERIMENT 7: SPLICING PREDICTION TRAINING
TRAINING CONFIGURATION:
  Timestamp:           2026-02-02T02:51:36.768142
  Experiment dir:      d:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7
  Data dir:            D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val
  Device:              cuda
  Seed:                42
  Batch size:          2048
  Learning rate:       0.001
  Weight decay:        0.0001
  Epochs:              50
  Patience:            7
  Dropout:             0.3
  Hidden dims:         [512, 256]
  Num classes:         3
  Embedding dim:       auto-detected

  ✅ Config.py copied to d:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7/config.py
Configuration saved to d:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7/config.json
Arguments saved to d:\Bio_sequence_Research_AITALAB\t

[Epoch 1] Train Loss=0.2622 | Val Loss=0.2341 | accuracy=0.9093 | precision=0.9111 | recall=0.9095 | specificity=0.9546 | f1=0.9100 | mcc=0.8643 | auc=0.9829 | balanced_acc=0.9095
  → New best model saved!


[Epoch 2] Train Loss=0.2223 | Val Loss=0.2229 | accuracy=0.9136 | precision=0.9132 | recall=0.9138 | specificity=0.9568 | f1=0.9133 | mcc=0.8706 | auc=0.9843 | balanced_acc=0.9138
  → New best model saved!


[Epoch 3] Train Loss=0.2107 | Val Loss=0.2160 | accuracy=0.9165 | precision=0.9165 | recall=0.9166 | specificity=0.9582 | f1=0.9166 | mcc=0.8748 | auc=0.9850 | balanced_acc=0.9166
  → New best model saved!


...

[Epoch 23] Train Loss=0.1463 | Val Loss=0.1455 | accuracy=0.9469 | precision=0.8992 | recall=0.8778 | specificity=0.9357 | f1=0.8881 | mcc=0.8268 | auc=0.9833 | balanced_acc=0.8778


[Epoch 24] Train Loss=0.1462 | Val Loss=0.1443 | accuracy=0.9473 | precision=0.9042 | recall=0.8734 | specificity=0.9337 | f1=0.8883 | mcc=0.8269 | auc=0.9834 | balanced_acc=0.8734


[Epoch 25] Train Loss=0.1460 | Val Loss=0.1447 | accuracy=0.9473 | precision=0.9111 | recall=0.8649 | specificity=0.9298 | f1=0.8866 | mcc=0.8254 | auc=0.9833 | balanced_acc=0.8649


[Epoch 26] Train Loss=0.1462 | Val Loss=0.1449 | accuracy=0.9469 | precision=0.8977 | recall=0.8801 | specificity=0.9366 | f1=0.8887 | mcc=0.8271 | auc=0.9834 | balanced_acc=0.8801


[Epoch 27] Train Loss=0.1461 | Val Loss=0.1447 | accuracy=0.9474 | precision=0.9139 | recall=0.8622 | specificity=0.9286 | f1=0.8865 | mcc=0.8252 | auc=0.9834 | balanced_acc=0.8622
Early stopping at epoch 27

EXPERIMENT 7 COMPLETED!
Results saved to: d:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7
Models saved in each set directory
Metrics and results saved as JSON
Model summaries saved as .txt files
Confusion matrices visualized in TensorBoard

To view TensorBoard logs:
   tensorboard --logdir d:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\ratio_*\set_*\tensorboard



### 3) Inference test dataset

In [3]:
from infer_test import run_inference_for_set

data_dir = args.data_dir
exp_dir = r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7'
device = args.device
batch_size = args.batch_size

ratio_folders = sorted(os.listdir(data_dir))
if args.ratio:
    ratio_folders = [r for r in ratio_folders if r == args.ratio]

all_results = []
for ratio in ratio_folders:
    ratio_path = os.path.join(data_dir, ratio)
    if not os.path.isdir(ratio_path):
        continue
    set_folders = sorted(os.listdir(ratio_path))
    if args.set:
        set_folders = [s for s in set_folders if s == args.set]

    for set_name in set_folders:
        set_path = os.path.join(ratio_path, set_name)
        if not os.path.isdir(set_path):
            continue
        print(f"\nProcessing {ratio}/{set_name}...")
        res = run_inference_for_set(data_dir, exp_dir, ratio, set_name, device, batch_size=batch_size)
        if res:
            all_results.append(res)

# summary
summary_path = os.path.join(exp_dir, 'inference_summary.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2)
print(f"\n[done] Inference completed. Summary saved to {summary_path}")

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Processing 1_1_1/set_1...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_1\test_results.json

Processing 1_1_1/set_10...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_10\test_results.json

Processing 1_1_1/set_2...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_2\test_results.json

Processing 1_1_1/set_3...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_3\test_results.json

Processing 1_1_1/set_4...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_4\test_results.json

Processing 1_1_1/set_5...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_5\test_results.j

In [6]:
from infer_test import run_inference_for_set
import matplotlib.pyplot as plt
import numpy as np
import itertools
from torch.utils.tensorboard import SummaryWriter

def plot_to_tensorboard(cm, class_names):
    """
    Tạo một Matplotlib Figure chứa Confusion Matrix.
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Tính toán tỉ lệ phần trăm để hiển thị (tùy chọn)
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, format(cm[i, j], 'd'), 
                 horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

data_dir = args.data_dir
exp_dir = r'D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7'
log_dir = os.path.join(exp_dir, 'tensorboard_test')
writer = SummaryWriter(log_dir=log_dir)
device = args.device
batch_size = args.batch_size
test_set = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv']

for test_data in test_set:
    # Tạo tên nhóm cho TensorBoard (ví dụ: test_10_1_1)
    test_tag = test_data.replace('.csv', '')
    
    ratio_folders = sorted(os.listdir(data_dir))
    if args.ratio:
        ratio_folders = [r for r in ratio_folders if r == args.ratio]

    all_results = []
    for ratio in ratio_folders:
        ratio_path = os.path.join(data_dir, ratio)
        if not os.path.isdir(ratio_path):
            continue
        set_folders = sorted(os.listdir(ratio_path))
        if args.set:
            set_folders = [s if s == args.set else None for s in set_folders] 

        for set_name in set_folders:
            if set_name is None: continue
            set_path = os.path.join(ratio_path, set_name)
            if not os.path.isdir(set_path):
                continue
                
            print(f"\nProcessing {ratio}/{set_name} on {test_data}...")
            res = run_inference_for_set(data_dir, exp_dir, ratio, set_name, test_data=test_data, device=device, batch_size=batch_size)
            
            if res:
                all_results.append(res)
                
                # 1. Log các chỉ số scalar (Accuracy, F1, AUPRC...)
                step = int(ratio.split('_')[-1]) if '_' in ratio else 0
                if 'metrics' in res:
                    for metric_name, value in res['metrics'].items():
                        # Kiểm tra nếu là giá trị số thì mới log
                        if isinstance(value, (int, float)):
                            writer.add_scalar(f'{metric_name}/{test_tag}', value, step)

                # 2. Log Confusion Matrix dưới dạng hình ảnh
                cm_array = np.array(res['confusion_matrix'])
                class_names = ['Class 0', 'Class 1', 'Class 2'] # Thay tên thật nếu bạn muốn
                
                fig = plot_to_tensorboard(cm_array, class_names)
                
                # tag: Confusion_Matrix/test_10_1_1, global_step: ratio (1, 2, 4, 10)
                writer.add_figure(f'Confusion_Matrix/{test_tag}', fig, global_step=step)
                
                plt.close(fig)

    # Summary JSON
    summary_path = os.path.join(exp_dir, test_data.replace('.csv', '_inference_summary.json'))
    with open(summary_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2)
    print(f"\n[done] Inference completed for {test_data}. Summary saved to {summary_path}")

writer.close()


Processing 1_1_1/set_1 on test_1_1_1.csv...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_1\test_1_1_1_results.json

Processing 1_1_1/set_10 on test_1_1_1.csv...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_10\test_1_1_1_results.json

Processing 1_1_1/set_2 on test_1_1_1.csv...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_2\test_1_1_1_results.json

Processing 1_1_1/set_3 on test_1_1_1.csv...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_3\test_1_1_1_results.json

Processing 1_1_1/set_4 on test_1_1_1.csv...
[saved] D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\training\trained_models\experiment_7\1_1_1\set_4\test_1_1_1_results.json

Processing 1_1_1/set_5 on test_1_1_1.csv..

### 4) View Experiment Results

Enter task1_splicing_prediction/training/trained_models/experiment_[number]/inference_summary.json

### 5) TensorBoard Monitoring

To monitor training progress in real-time:

```bash
tensorboard --logdir train/task1_splicing_prediction/training/trained_models/experiment_7/
```

Then open the URL in your browser to view:
- Training/Validation loss curves
- Metric evaluation (accuracy, precision, recall, F1, MCC, auc, balanced acc, specificity)
- Model architecture summary
